In [43]:
from sigcom.coding.atsc import code_param_long
from sigcom.tx.modcod import LdpcEncAtsc
from sigcom.cuda.g_v2c import GpuFecDec
import numpy as np
from numba import cuda

N_codewords = 200
CR = [8, 15]
N_ldpc = 64800
code = code_param_long.get(CR)

ldpcEncAtsc = LdpcEncAtsc(CR, N_ldpc)
#gpuFecDec = GpuFecDec(code, N_codewords)

ldpcEncAtsc.generate(N_codewords)

tx = 1-2*ldpcEncAtsc.codebits
noise = np.random.randn(len(tx))
P_noise = .5
rx = tx+noise*np.sqrt(P_noise)
Llrs = np.asarray(2/P_noise*rx, np.float32)
dLlrs = cuda.to_device(Llrs)

CF = code.nCyclicFactor
N_diags = code.N_diags
C2Vs = np.random.randn(N_diags*N_codewords, CF)
C2Vs = np.asarray(C2Vs, dtype=np.float32)
dC2Vs = cuda.to_device(C2Vs)
#gpuFecDec.dC2Vs = dC2Vs

N_max_fec_iterations = 1
# gpuFecDec.decode(dLlrs, N_max_fec_iterations)

In [45]:
%timeit dC2Vs = cuda.to_device(C2Vs)

134 ms ± 969 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit gpuFecDec.decode(dLlrs, N_max_fec_iterations)

51.3 ms ± 61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
hLlrs_ext = gpuFecDec.dLlrs_ext.copy_to_host()

In [6]:
hLlrs_ext

array([ 1.2842157 , -6.051565  ,  5.516615  , ...,  1.5176891 ,
       -4.0602818 ,  0.34357214], dtype=float32)

In [5]:
from sigcom.rx.cpu_v2c import update_v2c

In [6]:
Llrs_ext_ref = update_v2c(Llrs, C2Vs, code, N_codewords)

In [10]:
np.sum(np.abs(Llrs_ext_ref-hLlrs_ext)**2)

0.0

In [11]:
%timeit Llrs_ext_ref = update_v2c(Llrs, C2Vs, code, N_codewords)

3.75 s ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit gpuFecDec.decode(dLlrs, N_max_fec_iterations)

2.07 ms ± 54.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
3.75/2.07e-3

1811.594202898551

In [4]:
from sigcom.cuda.g_v2c_2 import GpuFecDec as GpuFecDec2
gFecDec2 = GpuFecDec2(code, N_codewords)
gFecDec2.dC2Vs = dC2Vs

In [5]:
gFecDec2.decode(dLlrs, N_max_fec_iterations)

In [23]:
%timeit hLlrs_ext2 = gFecDec2.dLlrs_ext.copy_to_host()
hLlrs_ext2

15.4 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


array([ 1.2842157 , -6.051565  ,  5.516615  , ...,  1.5176891 ,
       -4.0602818 ,  0.34357214], dtype=float32)

In [24]:
np.sum(np.abs(hLlrs_ext-hLlrs_ext2))

0.0

In [13]:
2e-3/192e-6

10.416666666666666

In [26]:
100*64800/15.4e-3/1e6

420.77922077922074

In [36]:
import time
Its = 100
t1 = time.clock()
for it in range(Its):
    gFecDec2.decode(dLlrs, N_max_fec_iterations)
    #gpuFecDec.decode(dLlrs, N_max_fec_iterations)
cuda.synchronize()
dt = time.clock() - t1
print(Its*N_ldpc*N_codewords/dt/1e6)
print(dt/Its/1e-3)

2073.085874703052
6.251550000000066


In [31]:
@cuda.jit
def func(dS):
    #for a,b in zip([1],[2]):
    #    dS[0] = a+b
    a=[1]
    dS[0]=np.array(a[0])
dS = cuda.device_array(1, dtype=np.int64)
func[1,1](dS)
hS = dS.copy_to_host()
hS

UntypedAttributeError: Failed at nopython (nopython frontend)
Unknown attribute 'array' of type Module(<module 'numpy' from '/home/peter/.local/lib/python3.6/site-packages/numpy/__init__.py'>)
File "<ipython-input-31-083c05b108b0>", line 6
[1] During: typing of get attribute at <ipython-input-31-083c05b108b0> (6)